In this python exercise, we will construct the Hamiltonian for the 2-site Ising model:
$$H = -J (\sigma_0^z \sigma_1^z) - h(\sigma_0^x + \sigma_1^x)$$
However, for numerics we always want to deal with unitless operators.  In this case, we actually look at $H/J$:
$$H/J = - (\sigma_0^z \sigma_1^z) - g(\sigma_0^x + \sigma_1^x)$$
We have already constructed the 4x4 matrix for this and found it to be:
$$-\left(\begin{array}{cc} J & h & h & 0 \\ h & -J & 0 & h\\ h & 0 & -J & h\\ 0 & h & h & J\end{array}\right)$$
or for $H/J$ in terms of $g$:
$$-\left(\begin{array}{cc} 1 & g & g & 0 \\ g & -1 & 0 & g\\ g & 0 & -1 & g\\ 0 & g & g & 1\end{array}\right)$$

We're going to solve this for many values of $g$, and we don't want to reenter the matrix for each one, so we will define a python function that takes in $g$ as an input, and gives back the matrix for that value of $g$.

A function can be defined as follows:

In [ ]:
def f(x):
    y = x**2 # ** is the exponential operator, so y is x squared
    return y

Now we can use this function:

In [ ]:
f(3)

Now we will construct our function for the Hamiltonian:

In [ ]:
import numpy as np

X = np.array( [[0,1,1,0],[1,0,0,1],[1,0,0,1],[0,1,1,0]] )# matrix for Sx0 + Sx1 here
ZZ = np.diag( [1,-1,-1,1] ) # matrix for Sz0 Sz1 here # Create a diagonal matrix with these elements on the diagonal

def get_H(g):
    H = -ZZ - g*X # function of g, ZZ, and X
    return H

Let's plot the eigenenergies as a function of $g$, from 0 to 2.  

We first generate a set of input value of $g$ with the ```np.linspace(start, stop, n)``` command.  This produces a vector containing ```n``` evenly spaced points starting at ```start``` and ending at ```stop```.

We then compute the energies for each $g$ from our input values, and store them in a table.  

Finally, we plot all the energies.  For this, we have to import another new package, ```pyplot```.  We then create a figure ```f``` with "axes" ```a```, and use the ```a.scatter(xs, ys)``` command to make the plot.

In [ ]:
num_pts = 21 # Use 21 points to get nice intervals of 0.1, since it includes the start and end
gs = np.linspace(0,2,num_pts) 
print(gs)

# Create an empty table to store the results:
energies = np.zeros( (4,num_pts) ) # This is a matrix of all zero, size is 4 (number of eigenvalues) x num_pts (# of pts)

for g_index, g in enumerate(gs): # This is a new kind of "for" loop. For every value in "gs", it stores that value in "g" and
                                 #  also stores where it was in the array (eg g=0 is at position 0, g=2 at position 20) and
                                 #  stores that in the variable "g_index"
    H = get_H(g)
    e, v = np.linalg.eigh(H) # Last time, we used "np.linalg.eig(M)".  The "h" on the end of "eigh" means that the function 
                             #   assumes "H" is Hermitian, and runs a little faster as a result
                             # e is a vector containing the eigenvalues, and v is a matrix whose columns are the
                             #   corresponding eigenvectors
    # Now let's sort them, from lowest e to highest e.  We won't worry about sorting v, since we won't be using it
    e = np.sort(e) # automatically goes from most negative to most positive
    
    # Store them in the table.  This length 4 vector goes into column "g_index":
    energies[:,g_index] = e

# import plotting package
import matplotlib.pyplot as plt

f, a = plt.subplots() # Create figure and "axes"
for row in range(4): # range(4) contains 4 numbers: 0,1,2,3, ie it is all integers starting at 0 and ending just before 4
    a.scatter(gs, energies[row]) # Add to the figure a scatter plot of the energies in the given row vs gs
# Label the figure:
a.set_xlabel('g', fontsize = 16);
a.set_ylabel('E', fontsize = 16);


Great! So far so good.  Now we will try to see some things about the ground state.  In particular, we are going to calculate four expectation values: $\langle \sigma_0^z + \sigma_1^z \rangle/2$, $\langle \sigma_0^z \sigma_1^z \rangle$, $\langle \sigma_0^x + \sigma_1^x \rangle/2$, and $\langle \sigma_0^x \sigma_1^x \rangle$.  To do this, we first find matrices for these operators, then do the same loop as above, in each finding the expectation value in the ground state.

In [ ]:
Z_avg_mat = np.diag([1,0,0,-1]) # Makes a diagonal matrix with these 4 elements on the diagonal
ZZ_mat = ZZ # We already made this one!
X_avg_mat = X/2 # This one too!
XX_mat = np.diag(np.ones(4))[::-1] # First put ones on the diagonal, then [::-1] reverses the order of the rows.

num_pts = 21
gs = np.linspace(0,2,num_pts)

# Create matrices to store the results
Z_avg_vals = np.zeros(num_pts)
ZZ_vals = np.zeros(num_pts)
X_avg_vals = np.zeros(num_pts)
XX_vals = np.zeros(num_pts)

for g_index, g in enumerate(gs):
    H = get_H(g)
    e, v = np.linalg.eigh(H)
    
    # Now we need to find which e is lowest, and get the corresponding v.  One way is to use the argsort function, like this:
    indices = np.argsort(e) # returns indices that say how it should be reordered if you wanted to sort
    e = e[indices] # this will reorder the rows.  e[np.array([1,3,2,4])] would swap elements 2 and 3 for example
    v = v[:, indices] # this reorders the columns, while leaving the rows constant
    
    GS = v[:,0]
    
    # Now our ground state is the first column of v, v[:,0]
    # We get expectation values using GS^dagger @ [operator to measure] @ GS
    Z_avg_vals[g_index] = GS.conj() @ Z_avg_mat @ GS
    ZZ_vals[g_index] = GS.conj() @ ZZ_mat @ GS
    X_avg_vals[g_index] = GS.conj() @ X_avg_mat @ GS
    XX_vals[g_index] = GS.conj() @ XX_mat @ GS
    
# Make plots
for name, values in [('<Z>',Z_avg_vals), ('<ZZ>',ZZ_vals), ('<X>',X_avg_vals), ('<XX>',XX_vals)]:
    f,a = plt.subplots()
    a.scatter(gs, values)
    a.set_xlabel('g', fontsize = 16);
    a.set_ylabel(name, fontsize = 16);
